# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')

c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [3]:
#Your code here; use a subquery. No join will be necessary.
query ="""select customerNumber,
                contactLastName,
                contactFirstName
            from customers 
            where customerNumber
            in ( select customerNumber 
                    from orders 
                    where orderDate='2003-01-31' 
                )
            """
df =pd.read_sql(query, conn)
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [4]:
pd.read_sql("""select * 
                    from orders 
                    where orderDate='2003-01-31'""",conn)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
pd.read_sql("""select * 
                    from products""",conn).columns

Index(['productCode', 'productName', 'productLine', 'productScale',
       'productVendor', 'productDescription', 'quantityInStock', 'buyPrice',
       'MSRP'],
      dtype='object')

In [45]:
query =""" 
            select p.productName,
                count(distinct od.orderNumber) as totalOrders, 
                sum(od.quantityOrdered) as totalsold_units
                from products as p 
                join orderdetails as od 
                using(productCode) 
                group by productCode 
                order by totalsold_units desc
            """
# c.execute(query)#run this to make the temporary table

df= pd.read_sql(query,conn)
df.head(10)

,productName,totalOrders,totalsold_units
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
5,1940s Ford truck,28,1061
6,1969 Harley Davidson Ultimate Chopper,28,1057
7,1957 Chevy Pickup,28,1056
8,1964 Mercedes Tour Bus,28,1053
9,1956 Porsche 356A Coupe,27,1052


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [7]:
#Your code here
query="""select p.productName,
                count(o.customerNumber) as total_people
            from products as p
            join orderdetails as od 
            on p.productCode = od.productCode
            join orders as o
            on od.orderNumber = o.orderNumber
            group by p.productName
            order by total_people desc
            """
df=pd.read_sql(query,conn)
df.head(10)

,productName,total_people
0,1992 Ferrari 360 Spider red,53
1,18th Century Vintage Horse Carriage,28
2,1900s Vintage Bi-Plane,28
3,1900s Vintage Tri-Plane,28
4,1913 Ford Model T Speedster,28
5,1917 Maxwell Touring Car,28
6,1926 Ford Fire Engine,28
7,1928 British Royal Navy Airplane,28
8,1928 Ford Phaeton Deluxe,28
9,1928 Mercedes-Benz SSK,28


temporary tables can be used to join multiple tables in very organized way

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [86]:
query =""" 
            select distinct e.employeeNumber, 
                    e.firstName, 
                    e.lastName, 
                    e.officeCode,
                    of.city
                    from employees as e
                    join offices as of
                    using(officeCode)
                    join customers as cus
                    on e.employeeNumber = cus.salesRepEmployeeNumber
                    join orders as o
                    using (customerNumber)
                    join orderdetails as od
                    using(orderNumber)
                    where od.productCode in (  select p.productCode 
                                                    from orders as o 
                                                    join orderdetails as od 
                                                    using(orderNumber)
                                                    join products as p
                                                    using (productCode)
                                                    group by p.productCode
                                                    having count(distinct o.customerNumber) <=20
                                                )
            """
pd.read_sql(query, conn)

,employeeNumber,firstName,lastName,officeCode,city
0,1370,Gerard,Hernandez,4,Paris
1,1401,Pamela,Castillo,4,Paris
2,1621,Mami,Nishi,5,Tokyo
3,1501,Larry,Bott,7,London
4,1337,Loui,Bondur,4,Paris
5,1286,Foon Yue,Tseng,3,NYC
6,1166,Leslie,Thompson,1,San Francisco
7,1165,Leslie,Jennings,1,San Francisco
8,1612,Peter,Marsh,6,Sydney
9,1188,Julie,Firrelli,2,Boston


In [72]:
#solution from the note
c.execute("""select distinct employeeNumber,
                officeCode,
                o.city,
                firstName,
                lastName
                from employees e
                join offices o
                using(officeCode)
                join customers c
                on e.employeeNumber = c.salesRepEmployeeNumber
                join orders
                using (customerNumber)
                join orderdetails
                using (orderNumber)
                    where productCode in (select productCode
                    from products
                    join orderdetails
                    using (productCode)
                    join orders
                    using (orderNumber)
                    group by 1
                    having count(distinct customerNumber) <= 20
                    );
                """
       )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur
5,1286,3,NYC,Foon Yue,Tseng
6,1166,1,San Francisco,Leslie,Thompson
7,1165,1,San Francisco,Leslie,Jennings
8,1612,6,Sydney,Peter,Marsh
9,1188,2,Boston,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [89]:
#Your code here

query="""select employeeNumber,
                lastName,
                firstName, 
                count(customerNumber) as customer_counts
            from employees as e
            join customers as cu
            on( e.employeeNumber = cu.salesRepEmployeeNumber)
                group by 1
                having avg(creditLimit)>15000;
            """
pd.read_sql(query, conn)

,employeeNumber,lastName,firstName,customer_counts
0,1165,Jennings,Leslie,6
1,1166,Thompson,Leslie,6
2,1188,Firrelli,Julie,6
3,1216,Patterson,Steve,6
4,1286,Tseng,Foon Yue,7
5,1323,Vanauf,George,8
6,1337,Bondur,Loui,6
7,1370,Hernandez,Gerard,7
8,1401,Castillo,Pamela,10
9,1501,Bott,Larry,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!